In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import re
import os
import patsy

from matplotlib.colors import ListedColormap

import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from esda.adbscan import ADBSCAN
from pysal.model import mgwr
from mgwr.sel_bw import Sel_BW

import random
random.seed(42)    # For reproducibility
np.random.seed(42) # For reproducibility

# Make numeric display a bit neater
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

In [2]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
plt.style.use('ggplot') # specifies that graphs should use ggplot styling
%matplotlib inline

In [4]:
tfont = {'fontname':'Liberation Sans Narrow', 'horizontalalignment':'left'}
import math
from geopandas import GeoDataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
import mcdm

In [3]:
pip install mcdm

     |████████████████████████████████| 69 kB 234 kB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 530 kB/s eta 0:00:01    |█████▍                          | 2.7 MB 2.9 MB/s eta 0:00:05     |█████▊                          | 2.8 MB 2.9 MB/s eta 0:00:05     |██████▊                         | 3.3 MB 2.9 MB/s eta 0:00:05     |███████▋                        | 3.7 MB 2.9 MB/s eta 0:00:05
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pointpats 2.2.0 requires opencv-contrib-python>=4.2.0, which is not installed.
access 1.1.1 requires Sphinx==2.4.3, which is not installed.
Note: you m

In [5]:
#加载数据
dg_area=gpd.read_file('final_output/dg/dg_area_withindex.shp',encoding="utf-8")
dg_station=gpd.read_file('final_output/dg/dg_station_withindex.shp',encoding="utf-8")
subway_grid=gpd.read_file('metro_data/mobile_grid/grid_subway_catchmentarea.shp',encoding="utf-8")
subway_flow=pd.read_csv('metro_data/mobile_grid/20190624.csv',low_memory=False,encoding="utf-8")

In [6]:
#提取站点area的grid
subway_grid2=subway_grid.copy()
subway_grid2.to_crs('EPSG:4326',inplace=True)
test = gpd.sjoin(subway_grid2, dg_area, how='left')
test.dropna(subset = ["id"], inplace=True)

In [7]:
#加载人流
subway_flow_69 = subway_flow[subway_flow["time"].isin([6, 9])]
subway_flow_69 = subway_flow_69.groupby(['o_grid','d_grid'])['num_total'].agg(flow_day_sum='sum').reset_index()

In [8]:
#O
subway_flow_O=subway_flow_69.copy()
subway_flow_O.rename(columns={'flow_day_sum':'O_flow','o_grid':'Tid'},inplace=True)
test1=pd.merge(test,subway_flow_O,on='Tid',how='left')
test_sub=test[['Tid']]
test_sub.rename(columns={'Tid':'d_grid'},inplace=True)
test2=pd.merge(test1,test_sub,on='d_grid',how='right')
O_dg_flow = test2.groupby(['id'])['O_flow'].agg(O_flw='sum').reset_index()
dg_area=pd.merge(dg_area,O_dg_flow,on='id',how='left')

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
subway_flow_D=subway_flow_69.copy()
subway_flow_D.rename(columns={'flow_day_sum':'D_flow','d_grid':'Tid'},inplace=True)
test1=pd.merge(test,subway_flow_D,on='Tid',how='left')
test_sub=test[['Tid']]
test_sub.rename(columns={'Tid':'o_grid'},inplace=True)
test2=pd.merge(test1,test_sub,on='o_grid',how='right')
D_dg_flow = test2.groupby(['id'])['D_flow'].agg(D_flw='sum').reset_index()
dg_area=pd.merge(dg_area,D_dg_flow,on='id',how='left')

In [10]:
dg_area

,id,P_pop_sum,road_lengt,area,line_name,station_na,lng,time,lat,lng84,...,S_communit,C_height_a,C_building,Node_index,Place_inde,Settlement,Container_,geometry,O_flw,D_flw
0,1,17962,"15,595.5288",1480994,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,?????,113.8602,15,23.0886,113.8552,...,1,15.8788,"528,990,969.9011",0.4376,0.2446,0.5303,0.1595,"POLYGON ((113.85222 23.10256, 113.85280 23.101...",57.0000,72.0000
1,2,12759,"23,735.6395",1864542,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,??,113.8444,15,23.0795,113.8394,...,0,11.3702,"1,109,945,989.8757",0.0000,0.2824,0.0160,0.2244,"POLYGON ((113.83738 23.09374, 113.83809 23.092...",38.0000,43.0000
2,3,28334,"30,091.6851",2053125,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,????,113.8142,15,23.0803,113.8091,...,2,11.8720,"3,667,108,211.4999",0.2984,0.3336,0.7398,0.5426,"POLYGON ((113.81015 23.09360, 113.81170 23.092...",84.0000,75.0000
3,4,50335,"34,708.3356",2760215,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,??,113.7883,15,23.0628,113.7832,...,2,10.3879,"4,052,113,742.7643",0.1431,0.6007,0.9145,0.4972,"POLYGON ((113.79216 23.07373, 113.79159 23.072...",257.0000,234.0000
4,5,56352,"24,674.7353",2535545,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,??,113.7787,15,23.0473,113.7735,...,1,11.8019,"4,368,756,146.9369",0.4521,0.5415,0.3458,0.4602,"POLYGON ((113.77052 23.06150, 113.77124 23.060...",586.0000,444.0000
5,6,48195,"27,694.4121",2192375,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,??,113.7764,15,23.0327,113.7712,...,1,14.2110,"4,728,564,040.6644",0.5248,0.4841,0.7269,0.6165,"POLYGON ((113.77324 23.04705, 113.77397 23.046...",772.0000,794.0000
6,7,32836,"25,336.0517",2043189,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,????,113.7677,15,23.0200,113.7625,...,0,13.8847,"3,006,041,866.9467",0.2793,0.4619,0.1505,0.4349,"POLYGON ((113.76953 23.03240, 113.76988 23.030...",441.0000,524.0000
7,8,25812,"36,398.8093",2559610,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,???,113.7592,15,23.0104,113.7540,...,2,18.0914,"2,911,627,253.9142",0.4514,0.4277,0.6650,0.5372,"POLYGON ((113.76198 23.02231, 113.76227 23.021...",483.0000,582.0000
8,9,24748,"29,630.2825",2647212,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,??,113.7469,15,22.9939,113.7417,...,1,20.8889,"2,086,775,393.0684",0.1567,0.4193,0.2731,0.4001,"POLYGON ((113.74870 23.00644, 113.74915 23.006...",349.0000,263.0000
9,10,18025,"23,793.5830",2152421,b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...,??,113.7289,15,22.9781,113.7237,...,0,12.4058,"2,244,902,478.4175",0.1198,0.2609,0.3176,0.2989,"POLYGON ((113.73072 22.98984, 113.73160 22.988...",221.0000,235.0000


In [11]:
dg_area.sum(axis = 0, skipna = True)
##该时段流动的总人口4278

id                                                          120
P_pop_sum                                                445176
road_lengt                                         383,059.9242
area                                                   31051694
line_name     b'\xe8\xbd\xa8\xe9\x81\x93\xe4\xba\xa4\xe9\x80...
station_na          ???????????????????????????????????????????
lng                                                  1,706.2261
time                                                        225
lat                                                    344.9801
lng84                                                1,706.1492
lat84                                                  345.0220
N_busStop_                                                 2070
N_Service_                                                   17
N_busline_                                              58.7360
N_external                                               5.7687
S_internal                              

In [12]:
dg_area['centricity_index']=(dg_area.D_flw-dg_area.O_flw)/4278

In [13]:
###计算functional polycentricity
subway_flow_O=subway_flow_69.copy()
subway_flow_O.rename(columns={'flow_day_sum':'O_flow','o_grid':'Tid'},inplace=True)
temp1=pd.merge(test,subway_flow_O,on='Tid',how='left')
temp_sub=test[['Tid','id']]
temp_sub.rename(columns={'Tid':'d_grid','id':'D_id'},inplace=True)
dg_within_flow=pd.merge(temp1,temp_sub,on='d_grid',how='right')
dg_flow = dg_within_flow.groupby(['id','D_id'])['O_flow'].agg(flw='sum').reset_index()

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
dg_flow

,id,D_id,flw
0,1.0000,1.0000,38.0000
1,1.0000,2.0000,2.0000
2,1.0000,3.0000,2.0000
3,1.0000,4.0000,2.0000
4,1.0000,5.0000,1.0000
5,1.0000,6.0000,2.0000
6,1.0000,7.0000,2.0000
7,1.0000,8.0000,3.0000
8,1.0000,10.0000,1.0000
9,1.0000,15.0000,4.0000


In [15]:
dg_area.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id                15 non-null     int64   
 1   P_pop_sum         15 non-null     int64   
 2   road_lengt        15 non-null     float64 
 3   area              15 non-null     int64   
 4   line_name         15 non-null     object  
 5   station_na        15 non-null     object  
 6   lng               15 non-null     float64 
 7   time              15 non-null     int64   
 8   lat               15 non-null     float64 
 9   lng84             15 non-null     float64 
 10  lat84             15 non-null     float64 
 11  N_busStop_        15 non-null     int64   
 12  N_Service_        15 non-null     int64   
 13  N_busline_        15 non-null     float64 
 14  N_external        15 non-null     float64 
 15  S_internal        15 non-null     float64 
 16  C_road_den        15

In [16]:
import networkx as nx
G_dg = nx.Graph()
G_dg = nx.from_pandas_edgelist(dg_flow, 'id', 'D_id',['flw'],create_using=nx.DiGraph)

In [17]:
#degree for each node
deg=nx.degree(G_dg,weight='flw')
deg

DiDegreeView({1.0: 129.0, 2.0: 81.0, 3.0: 159.0, 4.0: 491.0, 5.0: 1030.0, 6.0: 1566.0, 7.0: 965.0, 8.0: 1065.0, 10.0: 456.0, 15.0: 194.0, 12.0: 647.0, 9.0: 612.0, 11.0: 313.0, 13.0: 439.0, 14.0: 409.0})

In [18]:
dg_den=nx.density(G_dg)
dg_degree = pd.DataFrame(dict(
    DEGREE = dict(deg)))
dg_degree['id']=dg_degree.index.values
dg_area=pd.merge(dg_area,dg_degree,on='id',how='left')

In [19]:
from statistics import stdev
sample=(dg_area.DEGREE.min(),dg_area.DEGREE.max())
dg_FP=dg_den*(1-(dg_area.DEGREE.std())/(stdev(sample)))
dg_FP
###东莞functional polycentricity——————0.5588852551372082

0.5588852551372082

In [20]:
temp=dg_area[['centricity_index','DEGREE','id']]
dg_station=pd.merge(dg_station,temp,on='id',how='left')
dg_station.to_file("dg_station_withcentricity.shp")
dg_area.to_file("dg_area_withcentricity.shp")

In [21]:
#加载数据
sz_area=gpd.read_file('final_output/sz/sz_area_withindex.shp',encoding="utf-8")
sz_station=gpd.read_file('final_output/sz/sz_station_withindex.shp',encoding="utf-8")

In [22]:
#提取站点area的grid
subway_grid2=subway_grid.copy()
subway_grid2.to_crs('EPSG:4326',inplace=True)
test = gpd.sjoin(subway_grid2, sz_area, how='left')
test.dropna(subset = ["id"], inplace=True)

In [23]:
#加载人流
subway_flow_69 = subway_flow[subway_flow["time"].isin([6, 9])]
subway_flow_69 = subway_flow_69.groupby(['o_grid','d_grid'])['num_total'].agg(flow_day_sum='sum').reset_index()

In [24]:
#O
subway_flow_O=subway_flow_69.copy()
subway_flow_O.rename(columns={'flow_day_sum':'O_flow','o_grid':'Tid'},inplace=True)
test1=pd.merge(test,subway_flow_O,on='Tid',how='left')
test_sub=test[['Tid']]
test_sub.rename(columns={'Tid':'d_grid'},inplace=True)
test2=pd.merge(test1,test_sub,on='d_grid',how='right')
O_sz_flow = test2.groupby(['id'])['O_flow'].agg(O_flw='sum').reset_index()
sz_area=pd.merge(sz_area,O_sz_flow,on='id',how='left')

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
subway_flow_D=subway_flow_69.copy()
subway_flow_D.rename(columns={'flow_day_sum':'D_flow','d_grid':'Tid'},inplace=True)
test1=pd.merge(test,subway_flow_D,on='Tid',how='left')
test_sub=test[['Tid']]
test_sub.rename(columns={'Tid':'o_grid'},inplace=True)
test2=pd.merge(test1,test_sub,on='o_grid',how='right')
D_sz_flow = test2.groupby(['id'])['D_flow'].agg(D_flw='sum').reset_index()
sz_area=pd.merge(sz_area,D_sz_flow,on='id',how='left')
sz_area['D_flw'] = sz_area['D_flw'].fillna(0)

In [26]:
sz_area

,id,P_pop_sum,road_lengt,line_name,station_na,lng,time,area,lat,lng84,...,S_communit,C_height_a,C_building,Node_index,Place_inde,Settlement,Container_,geometry,O_flw,D_flw
0,1,19195,0.3588,??1??(???)(??--???),??,114.1187,15,"1,645,538.4390",22.5321,114.1136,...,2,26.9008,"2,739,953,307.6534",0.1961,0.4189,0.1034,0.6234,"POLYGON ((114.11357 22.54607, 114.11525 22.545...","6,902.0000","10,600.0000"
1,2,45408,0.7145,??1??(???)(??--???),??,114.1189,15,"3,369,940.4210",22.5397,114.1138,...,8,24.4371,"3,154,999,745.2876",0.2628,0.4429,0.3718,0.6432,"POLYGON ((114.11481 22.55262, 114.11581 22.551...","13,334.0000","18,525.0000"
2,3,42492,0.6562,??1??(???)(??--???),??,114.1169,15,"3,272,737.5670",22.5442,114.1118,...,6,22.5534,"3,339,467,027.6266",0.3025,0.4540,0.3348,0.6358,"POLYGON ((114.11584 22.55701, 114.11784 22.555...","13,727.0000","17,359.0000"
3,4,87540,0.7224,??1??(???)(??--???),???,114.1078,15,"3,183,149.1720",22.5418,114.1027,...,6,22.9828,"2,855,648,439.3592",0.2327,0.5160,0.3560,0.6312,"POLYGON ((114.10670 22.55365, 114.10770 22.553...","13,499.0000","18,268.0000"
4,5,99011,0.6267,??1??(???)(??--???),???,114.0952,15,"3,157,301.4840",22.5406,114.0900,...,9,22.3274,"3,237,672,542.5516",0.2556,0.5231,0.4678,0.6225,"POLYGON ((114.09105 22.55413, 114.09341 22.552...","13,522.0000","22,900.0000"
5,6,58213,0.7238,??1??(???)(??--???),???,114.0851,15,"3,298,703.4260",22.5405,114.0800,...,10,22.2006,"3,134,665,384.7216",0.2209,0.4753,0.5430,0.6430,"POLYGON ((114.08200 22.55345, 114.08517 22.550...","15,550.0000","24,716.0000"
6,7,131025,1.0234,??1??(???)(??--???),??,114.0682,15,"3,392,866.9790",22.5349,114.0631,...,2,25.1085,"2,459,753,929.2875",0.2377,0.5470,0.3061,0.7121,"POLYGON ((114.06307 22.54932, 114.06372 22.548...","17,300.0000","25,130.0000"
7,8,148984,1.0137,??1??(???)(??--???),????,114.0611,15,"3,228,908.8330",22.5348,114.0560,...,1,25.5482,"2,720,156,559.2282",0.2463,0.5594,0.2881,0.7561,"POLYGON ((114.05798 22.54770, 114.06022 22.545...","16,695.0000","28,200.0000"
8,9,159188,0.9705,??1??(???)(??--???),????,114.0546,15,"3,036,664.9600",22.5347,114.0495,...,2,22.2778,"2,340,954,223.2824",0.2254,0.5084,0.3073,0.7158,"POLYGON ((114.05147 22.54763, 114.05362 22.545...","14,526.0000","24,564.0000"
9,10,150242,0.5509,??1??(???)(??--???),???,114.0386,15,"2,283,581.4250",22.5389,114.0336,...,2,21.4123,"1,855,579,265.8959",0.1703,0.5334,0.1806,0.5528,"POLYGON ((114.03355 22.55165, 114.03455 22.551...","7,265.0000","12,504.0000"


In [27]:
sz_area.sum(axis = 0, skipna = True)
##该时段流动的总人口1221536

id                                                        24531
P_pop_sum                                              12499046
road_lengt                                             101.7720
line_name     ??1??(???)(??--???)??1??(???)(??--???)??1??(??...
station_na    ??????????????????????????????????????????????...
lng                                                 25,200.7431
time                                                       3315
area                                           549,577,092.1190
lat                                                  4,991.1095
lng84                                               25,199.6353
lat84                                                4,991.7340
N_busStop_                                                64849
N_service_                                               221310
N_Servic_1                                                  267
N_busline_                                           1,081.8956
N_external                              

In [28]:
sz_area['centricity_index']=(sz_area.D_flw-sz_area.O_flw)/1221536

In [29]:
###计算functional polycentricity
subway_flow_O=subway_flow_69.copy()
subway_flow_O.rename(columns={'flow_day_sum':'O_flow','o_grid':'Tid'},inplace=True)
temp1=pd.merge(test,subway_flow_O,on='Tid',how='left')
temp_sub=test[['Tid','id']]
temp_sub.rename(columns={'Tid':'d_grid','id':'D_id'},inplace=True)
sz_within_flow=pd.merge(temp1,temp_sub,on='d_grid',how='right')
sz_flow = sz_within_flow.groupby(['id','D_id'])['O_flow'].agg(flw='sum').reset_index()


/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
sz_flow

,id,D_id,flw
0,1.0000,1.0000,192.0000
1,1.0000,2.0000,256.0000
2,1.0000,3.0000,259.0000
3,1.0000,4.0000,220.0000
4,1.0000,5.0000,178.0000
...,...,...,...
40964,221.0000,216.0000,4.0000
40965,221.0000,218.0000,20.0000
40966,221.0000,219.0000,29.0000
40967,221.0000,220.0000,45.0000


In [31]:
sz_area.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 221 entries, 0 to 220
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id                221 non-null    int64   
 1   P_pop_sum         221 non-null    int64   
 2   road_lengt        221 non-null    float64 
 3   line_name         221 non-null    object  
 4   station_na        221 non-null    object  
 5   lng               221 non-null    float64 
 6   time              221 non-null    int64   
 7   area              221 non-null    float64 
 8   lat               221 non-null    float64 
 9   lng84             221 non-null    float64 
 10  lat84             221 non-null    float64 
 11  N_busStop_        221 non-null    int64   
 12  N_service_        221 non-null    int64   
 13  N_Servic_1        221 non-null    int64   
 14  N_busline_        221 non-null    float64 
 15  N_external        221 non-null    float64 
 16  S_internal        

In [32]:
import networkx as nx
G_sz = nx.Graph()
G_sz = nx.from_pandas_edgelist(sz_flow, 'id', 'D_id',['flw'],create_using=nx.DiGraph)

In [33]:
#degree for each node
deg=nx.degree(G_sz,weight='flw')
deg

DiDegreeView({1.0: 17502.0, 2.0: 31859.0, 3.0: 31086.0, 4.0: 31767.0, 5.0: 36422.0, 6.0: 40266.0, 7.0: 42430.0, 8.0: 44895.0, 9.0: 39090.0, 10.0: 19769.0, 11.0: 17218.0, 12.0: 11917.0, 13.0: 5584.0, 14.0: 8107.0, 15.0: 13578.0, 16.0: 17733.0, 17.0: 23659.0, 18.0: 25416.0, 19.0: 23242.0, 20.0: 17237.0, 21.0: 5660.0, 22.0: 3958.0, 23.0: 8419.0, 24.0: 11937.0, 25.0: 8254.0, 26.0: 15416.0, 27.0: 15807.0, 28.0: 7343.0, 29.0: 3333.0, 30.0: 1915.0, 31.0: 995.0, 32.0: 390.0, 33.0: 2364.0, 34.0: 4046.0, 35.0: 2162.0, 36.0: 2340.0, 37.0: 4158.0, 38.0: 3587.0, 39.0: 2797.0, 40.0: 4551.0, 41.0: 11042.0, 42.0: 21877.0, 43.0: 40308.0, 44.0: 40240.0, 45.0: 33208.0, 46.0: 43487.0, 47.0: 31953.0, 48.0: 27969.0, 49.0: 17753.0, 50.0: 9527.0, 51.0: 5881.0, 52.0: 8088.0, 53.0: 4509.0, 54.0: 1649.0, 55.0: 9585.0, 56.0: 7688.0, 57.0: 25710.0, 58.0: 13265.0, 59.0: 14857.0, 60.0: 11330.0, 61.0: 8705.0, 62.0: 9741.0, 63.0: 10711.0, 64.0: 7421.0, 65.0: 4207.0, 66.0: 1101.0, 67.0: 12306.0, 68.0: 17783.0, 69.0: 28

In [34]:
sz_den=nx.density(G_sz)
sz_degree = pd.DataFrame(dict(
    DEGREE = dict(deg)))
sz_degree['id']=sz_degree.index.values
sz_area=pd.merge(sz_area,sz_degree,on='id',how='left')

In [35]:
from statistics import stdev
sample=(sz_area.DEGREE.min(),sz_area.DEGREE.max())
sz_FP=sz_den*(1-(sz_area.DEGREE.std())/(stdev(sample)))
sz_FP
###深圳functional polycentricity——————0.5663215997878089

0.5663215997878089

In [36]:
temp=sz_area[['centricity_index','DEGREE','id']]
sz_station=pd.merge(sz_station,temp,on='id',how='left')
sz_station.to_file("sz_station_withcentricity.shp")
sz_area.to_file("sz_area_withcentricity.shp")

In [37]:
#加载数据
gz_area=gpd.read_file('final_output/gz/gz_area_withindex.shp',encoding="utf-8")
gz_station=gpd.read_file('final_output/gz/gz_station_withindex.shp',encoding="utf-8")

In [38]:
#提取站点area的grid
subway_grid2=subway_grid.copy()
subway_grid2.to_crs('EPSG:4326',inplace=True)
test = gpd.sjoin(subway_grid2, gz_area, how='left')
test.dropna(subset = ["id"], inplace=True)

In [39]:
#加载人流
subway_flow_69 = subway_flow[subway_flow["time"].isin([6, 9])]
subway_flow_69 = subway_flow_69.groupby(['o_grid','d_grid'])['num_total'].agg(flow_day_sum='sum').reset_index()

In [40]:
#O
subway_flow_O=subway_flow_69.copy()
subway_flow_O.rename(columns={'flow_day_sum':'O_flow','o_grid':'Tid'},inplace=True)
test1=pd.merge(test,subway_flow_O,on='Tid',how='left')
test_sub=test[['Tid']]
test_sub.rename(columns={'Tid':'d_grid'},inplace=True)
test2=pd.merge(test1,test_sub,on='d_grid',how='right')
O_gz_flow = test2.groupby(['id'])['O_flow'].agg(O_flw='sum').reset_index()
gz_area=pd.merge(gz_area,O_gz_flow,on='id',how='left')

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [41]:
subway_flow_D=subway_flow_69.copy()
subway_flow_D.rename(columns={'flow_day_sum':'D_flow','d_grid':'Tid'},inplace=True)
test1=pd.merge(test,subway_flow_D,on='Tid',how='left')
test_sub=test[['Tid']]
test_sub.rename(columns={'Tid':'o_grid'},inplace=True)
test2=pd.merge(test1,test_sub,on='o_grid',how='right')
D_gz_flow = test2.groupby(['id'])['D_flow'].agg(D_flw='sum').reset_index()
gz_area=pd.merge(gz_area,D_gz_flow,on='id',how='left')
gz_area['D_flw'] = gz_area['D_flw'].fillna(0)

In [42]:
gz_area.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          200 non-null    int64   
 1   lng84       200 non-null    float64 
 2   lat84       200 non-null    float64 
 3   area        200 non-null    int64   
 4   line_name   200 non-null    object  
 5   station_na  200 non-null    object  
 6   lng         200 non-null    object  
 7   lat         200 non-null    object  
 8   N_busStop_  200 non-null    int64   
 9   N_service_  200 non-null    int64   
 10  N_servic_1  200 non-null    int64   
 11  P_poi_Dive  200 non-null    float64 
 12  S_communit  200 non-null    int64   
 13  C_height_a  200 non-null    float64 
 14  road_lengt  200 non-null    float64 
 15  N_busLine_  200 non-null    float64 
 16  C_building  200 non-null    float64 
 17  N_external  200 non-null    float64 
 18  S_internal  200 non-null    float64 
 19  

In [43]:
gz_area.sum(axis = 0, skipna = True)
##该时段流动的总人口1,273,931

id                                                        20100
lng84                                               22,672.9039
lat84                                                4,627.5890
area                                                  397464400
line_name     ??1??(??--????)??1??(??--????)??1??(??--????)?...
station_na    ??????????????????????????????????????????????...
lng           113.231978113.232498113.23403113.235703113.239...
lat           23.06543723.07887823.08705523.09859223.1105242...
N_busStop_                                                49020
N_service_                                               202635
N_servic_1                                                  236
P_poi_Dive                                             139.3356
S_communit                                                  826
C_height_a                                           4,543.5646
road_lengt                                       7,468,804.0104
N_busLine_                              

In [44]:
gz_area['centricity_index']=(gz_area.D_flw-gz_area.O_flw)/1273931

In [45]:
###计算functional polycentricity
subway_flow_O=subway_flow_69.copy()
subway_flow_O.rename(columns={'flow_day_sum':'O_flow','o_grid':'Tid'},inplace=True)
temp1=pd.merge(test,subway_flow_O,on='Tid',how='left')
temp_sub=test[['Tid','id']]
temp_sub.rename(columns={'Tid':'d_grid','id':'D_id'},inplace=True)
gz_within_flow=pd.merge(temp1,temp_sub,on='d_grid',how='right')
gz_flow = gz_within_flow.groupby(['id','D_id'])['O_flow'].agg(flw='sum').reset_index()


In [46]:
gz_flow

,id,D_id,flw
0,1.0000,1.0000,68.0000
1,1.0000,2.0000,105.0000
2,1.0000,3.0000,141.0000
3,1.0000,4.0000,113.0000
4,1.0000,5.0000,130.0000
...,...,...,...
26329,200.0000,196.0000,464.0000
26330,200.0000,197.0000,413.0000
26331,200.0000,198.0000,407.0000
26332,200.0000,199.0000,162.0000


In [47]:
import networkx as nx
G_gz = nx.Graph()
G_gz = nx.from_pandas_edgelist(gz_flow, 'id', 'D_id',['flw'],create_using=nx.DiGraph)

In [48]:
#degree for each node
deg=nx.degree(G_gz,weight='flw')
deg

DiDegreeView({1.0: 6555.0, 2.0: 7916.0, 3.0: 10547.0, 4.0: 9220.0, 5.0: 17296.0, 6.0: 27845.0, 7.0: 34469.0, 8.0: 52188.0, 9.0: 56125.0, 10.0: 57399.0, 11.0: 41954.0, 12.0: 36332.0, 13.0: 47563.0, 14.0: 62300.0, 15.0: 55328.0, 16.0: 23932.0, 17.0: 1685.0, 18.0: 5270.0, 19.0: 21886.0, 20.0: 17387.0, 21.0: 5155.0, 23.0: 10637.0, 24.0: 36959.0, 25.0: 44324.0, 26.0: 58442.0, 27.0: 52731.0, 28.0: 12908.0, 29.0: 19877.0, 30.0: 7102.0, 31.0: 3895.0, 32.0: 6258.0, 33.0: 8502.0, 34.0: 3546.0, 35.0: 4555.0, 37.0: 49046.0, 38.0: 16400.0, 39.0: 12891.0, 40.0: 14459.0, 41.0: 11274.0, 42.0: 7100.0, 43.0: 9587.0, 45.0: 3005.0, 46.0: 1175.0, 47.0: 3511.0, 48.0: 3453.0, 50.0: 454.0, 53.0: 790.0, 54.0: 931.0, 58.0: 1111.0, 64.0: 176.0, 67.0: 2537.0, 68.0: 1499.0, 69.0: 6010.0, 70.0: 9941.0, 71.0: 18960.0, 72.0: 11401.0, 77.0: 4861.0, 79.0: 22711.0, 80.0: 24165.0, 81.0: 28604.0, 82.0: 45708.0, 83.0: 55424.0, 84.0: 41531.0, 85.0: 29693.0, 86.0: 31672.0, 87.0: 30356.0, 88.0: 17209.0, 89.0: 14697.0, 90.0: 1

In [49]:
gz_den=nx.density(G_gz)
gz_degree = pd.DataFrame(dict(
    DEGREE = dict(deg)))
gz_degree['id']=gz_degree.index.values
gz_area=pd.merge(gz_area,gz_degree,on='id',how='left')

In [50]:
from statistics import stdev
sample=(gz_area.DEGREE.min(),gz_area.DEGREE.max())
gz_FP=gz_den*(1-(gz_area.DEGREE.std())/(stdev(sample)))
gz_FP
###广州functional polycentricity——————0.4222301409712362

0.4222301409712362

In [51]:
temp=gz_area[['centricity_index','DEGREE','id']]
gz_station=pd.merge(gz_station,temp,on='id',how='left')
gz_station.to_file("gz_station_withcentricity.shp")
gz_area.to_file("gz_area_withcentricity.shp")

In [5]:
gz_station=gpd.read_file('final_output/gz/gz_station_withcentricity.shp',encoding="utf-8")

In [6]:
gz_station.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   POLY_ID     200 non-null    float64 
 1   line_name   200 non-null    object  
 2   station_na  200 non-null    object  
 3   lng         200 non-null    float64 
 4   lat         200 non-null    float64 
 5   lng84       200 non-null    float64 
 6   lat84       200 non-null    float64 
 7   id          200 non-null    int64   
 8   Node_index  200 non-null    float64 
 9   Place_inde  200 non-null    float64 
 10  Settlement  200 non-null    float64 
 11  Container_  200 non-null    float64 
 12  centricity  200 non-null    float64 
 13  DEGREE      200 non-null    float64 
 14  geometry    200 non-null    geometry
dtypes: float64(11), geometry(1), int64(1), object(2)
memory usage: 23.6+ KB


In [7]:
gz_polycentricity_cor=gz_station[['Node_index','Place_inde','Settlement','Container_','centricity']]

In [8]:
gz_polycentricity_cor.rename(columns={'Node_index':'Node','Place_inde':'Place','Container_':'Container','centricity':'Mobility',},inplace=True)

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
gz_polycentricity_cor.corr()

,Node,Place,Settlement,Container,Mobility
Node,1.0000,0.2232,0.3360,0.3876,-0.0190
Place,0.2232,1.0000,0.5357,0.4106,0.0954
Settlement,0.3360,0.5357,1.0000,0.3728,0.0117
Container,0.3876,0.4106,0.3728,1.0000,0.2038
Mobility,-0.0190,0.0954,0.0117,0.2038,1.0000


In [10]:
sz_station=gpd.read_file('final_output/sz/sz_station_withcentricity.shp',encoding="utf-8")
sz_polycentricity_cor=sz_station[['Node_index','Place_inde','Settlement','Container_','centricity']]
sz_polycentricity_cor.corr()

,Node_index,Place_inde,Settlement,Container_,centricity
Node_index,1.0000,0.2206,0.1035,0.3188,0.0783
Place_inde,0.2206,1.0000,0.0025,0.3353,0.0847
Settlement,0.1035,0.0025,1.0000,0.5150,0.1789
Container_,0.3188,0.3353,0.5150,1.0000,0.2370
centricity,0.0783,0.0847,0.1789,0.2370,1.0000


In [11]:
fs_station=gpd.read_file('final_output/fs/fs_station_withcentricity.shp',encoding="utf-8")
fs_polycentricity_cor=fs_station[['Node_index','Place_inde','Settlement','Container_','centricity']]
fs_polycentricity_cor.corr()

,Node_index,Place_inde,Settlement,Container_,centricity
Node_index,1.0000,0.6131,0.3828,-0.0599,0.0313
Place_inde,0.6131,1.0000,0.3514,-0.0033,0.1229
Settlement,0.3828,0.3514,1.0000,0.5115,-0.0396
Container_,-0.0599,-0.0033,0.5115,1.0000,0.0368
centricity,0.0313,0.1229,-0.0396,0.0368,1.0000


In [12]:
dg_station=gpd.read_file('final_output/dg/dg_station_withcentricity.shp',encoding="utf-8")
dg_polycentricity_cor=dg_station[['Node_index','Place_inde','Settlement','Container_','centricity']]
dg_polycentricity_cor.corr()

,Node_index,Place_inde,Settlement,Container_,centricity
Node_index,1.0000,0.1510,0.2327,0.4496,0.1571
Place_inde,0.1510,1.0000,0.2902,0.4961,-0.2668
Settlement,0.2327,0.2902,1.0000,0.4463,-0.0349
Container_,0.4496,0.4961,0.4463,1.0000,-0.0463
centricity,0.1571,-0.2668,-0.0349,-0.0463,1.0000


In [13]:
gba_station=gpd.read_file('final_output/gba_new/gba_station_withcentricity.shp',encoding="utf-8")
gba_polycentricity_cor=gba_station[['Node_index','Place_inde','centricity']]
gba_polycentricity_cor.corr()

,Node_index,Place_inde,centricity
Node_index,1.0000,-0.0309,-0.1713
Place_inde,-0.0309,1.0000,-0.0503
centricity,-0.1713,-0.0503,1.0000
